In [1]:
# ML_in_Finance-1D-CNNs
# Author: Matthew Dixon
# Version: 1.0 (24.7.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6 and Tensorflow 1.3.0
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Bilokon P., Dixon M.F. and I. Halperin, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 

# Using Keras to implement a 1D convolutional neural network (CNN) for timeseries prediction.

In [2]:
import numpy as np

from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential

Using TensorFlow backend.


In [72]:
np.set_printoptions(threshold=25)

### Creating the 1D CNN

We create a simple convolutional neural network: a 1D convolutional layer, followed by a dense layer.

It will allow us to predict the next value in a timeseries given an input sequence of length `window_size`

The `filter_length` is the length (in time-steps) of the sliding window that gets convolved with each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
`nb_filter` is the number of such filters to learn (roughly, input patterns to recognize).

The model can handle multivariate timeseries (with `nb_input_series` variables) and multiple (`nb_outputs`) prediction targets. Predicting future values of a timeseries means setting these equal to one another.

In [41]:
def make_CNN(window_size, filter_length,  nb_filter=4, nb_input_series=1, nb_outputs=1):
    """
    window_size (int): number of observations in each input sequence
    filter length (int): length of the convolutional layer's filters
    nb_filter (int): number of filters learned in the convolutional layer
    nb_input_series (int): number of features of the input timeseries (1 for a univariate timeseries)
    nb_outputs (int): number of features being predicted (equal to nb_input_series 
        for predicting a timeseries at a set horizon)"""
    
    model = Sequential((
        # The convolutional layer learns `nb_filter` filters (aka kernels), 
        # each of size `(filter_length, nb_input_series)`.  
        # Its output will have shape `(None, window_size - filter_length + 1, nb_filter)` ,  
        # i.e., for each position in the input timeseries, the activation of each filter at that position.
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', input_shape=(window_size, 1)),
        Flatten(),
        Dense(1, activation='linear'), # For classification, a 'sigmoid' activation function would be used
    ))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    return model

In [81]:
CNN_model = make_CNN(window_size=50, filter_length=5, nb_filter=4)

In [44]:
print('input shape:', CNN_model.layers[0].input_shape)
print('output shape:', CNN_model.layers[-1].output_shape)

input shape: (None, 50, 1)
output shape: (None, 1)


### Data preparation
  
We define a function to format a timeseries for training the neural network. It creates corresponding arrays of input sequences `X` and output values `y`. They have the same length as each other; the remaining dimensions must match the input and output layers of the model respectively:

The `X` input to the model's `fit()` method should be a 3D array of shape `(n_instances, window_size, n_ts_variables)`; each instance being a 2D array of shape `(window_size, nb_input_series)`.  For example, for `window_size = 3` and `nb_input_series = 1` (a univariate timeseries), one instance could be `[[0], [1], [2]]`

For each input instance, the output is a vector of size `nb_outputs`, usually the value(s) predicted to come after the last value in that input instance, i.e., the next value in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``. 


In [45]:
def make_timeseries_instances(timeseries, window_size):
    # Convert 1D vectors to 2D column vectors
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T 

    if not 0 < window_size < timeseries.shape[0]:
        raise ValueError('Please set 0 < window size < timeseries length')
    
    # `X `is the tensor containing the inputs for the model
    # each row of `X` is a sequence of `window_size` observations from the timeseries
    X = [timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]
    
    # for training the model, the array's dimensions must match the input layer of the CNN
    # that is, a 3D array of shape (timeseries.shape[0] - window_size, window_size, nof_ts_variables)
    X = np.atleast_3d(np.array(X))
    
    # For each row of `X`, the corresponding row of `y` is the 
    # desired output -- in this case, the subsequent value in the timeseries 
    y = timeseries[window_size:]
    
    return X, y

For example:  

In [46]:
X_fib, y_fib = make_timeseries_instances([1,1,2,3,5,8,13,21], 5)
print('X:', X_fib, 'y:', y_fib, sep='\n')

X:
[[[ 1]
  [ 1]
  [ 2]
  [ 3]
  [ 5]]

 [[ 1]
  [ 2]
  [ 3]
  [ 5]
  [ 8]]

 [[ 2]
  [ 3]
  [ 5]
  [ 8]
  [13]]]
y:
[[ 8]
 [13]
 [21]]


Create a toy timeseries and split it for training and testing the CNN:

In [47]:
timeseries = np.arange(1000) # the timeseries f(t)

In [58]:
X, y = make_timeseries_instances(timeseries, window_size=50)

test_ratio = 0.01 # In real life you'd usually want to use 0.2 - 0.5
test_size = int(test_ratio * len(timeseries)) 

# the "most recent" values are used for testing the model to avoid look-ahead bias
X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]

In [59]:
[__.shape for __ in [X_train, X_test, y_train, y_test]]

[(940, 50, 1), (10, 50, 1), (940, 1), (10, 1)]

In [62]:
X_test[0]

array([[940],
       [941],
       [942],
       ...,
       [987],
       [988],
       [989]])

In [63]:
y_test[0]

array([990])

### Training the model

Note that `validation_data` is not used to train the model, but allows you to monitor its out-of-sample performance during training

In [50]:
CNN_model.fit(X_train, y_train, epochs=25, batch_size=2, validation_data=(X_test, y_test))

Train on 940 samples, validate on 10 samples
Epoch 1/25
940/940 [==============================] - 5s 5ms/step - loss: 343281.4171 - mae: 519.2769 - val_loss: 988153.0875 - val_mae: 994.0548
Epoch 2/25
940/940 [==============================] - 3s 3ms/step - loss: 342824.9544 - mae: 518.8339 - val_loss: 987283.2500 - val_mae: 993.6171
Epoch 3/25
940/940 [==============================] - 3s 3ms/step - loss: 342368.0035 - mae: 518.4000 - val_loss: 986404.0750 - val_mae: 993.1746
Epoch 4/25
940/940 [==============================] - 3s 3ms/step - loss: 341912.2947 - mae: 517.9519 - val_loss: 985534.8750 - val_mae: 992.7369
Epoch 5/25
940/940 [==============================] - 3s 3ms/step - loss: 341457.3739 - mae: 517.5190 - val_loss: 984660.2500 - val_mae: 992.2963
Epoch 6/25
940/940 [==============================] - 3s 3ms/step - loss: 341002.9669 - mae: 517.0750 - val_loss: 983791.4625 - val_mae: 991.8585
Epoch 7/25
940/940 [==============================] - 3s 3ms/step - loss: 34054

We can inspect the weights of the convolutional layer:

In [103]:
CNN_model.layers[0].get_weights()

[array([[[-0.3409522 ,  0.16252264, -0.06636527, -0.4413616 ]],
 
        [[ 0.38766423, -0.01832604,  0.37046096, -0.3154464 ]],
 
        [[-0.30810523,  0.04050711,  0.00164559, -0.43126032]],
 
        [[ 0.4819189 ,  0.3402339 ,  0.02723256, -0.30651313]],
 
        [[-0.36740988,  0.06383643,  0.11094108,  0.21941969]]],
       dtype=float32),
 array([0., 0., 0., 0.], dtype=float32)]

### Making predictions with the model
Get the predicted values for the test set:

In [104]:
y_pred = CNN_model.predict(X_test)

In [73]:
print(np.column_stack((y_test, y_pred)))

[[ 990.          991.45458984]
 [ 991.          992.45617676]
 [ 992.          993.45770264]
 [ 993.          994.45935059]
 [ 994.          995.46081543]
 [ 995.          996.46252441]
 [ 996.          997.46411133]
 [ 997.          998.46569824]
 [ 998.          999.46740723]
 [ 999.         1000.46905518]]
